In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/debt-default-prediction/X_test.csv
/kaggle/input/debt-default-prediction/DataDictionary.xlsx
/kaggle/input/debt-default-prediction/valid.csv
/kaggle/input/debt-default-prediction/train.csv


# Train dataset is read and the rows with missing target value is removed from the dataset.

In [2]:
X_train = pd.read_csv('/kaggle/input/debt-default-prediction/train.csv')
X_train.dropna(axis=0, subset=['loan_status'], inplace=True)
X_train.head()
y_train = X_train.loan_status
X_train.drop(['loan_status'], axis = 1,inplace= True )


In [3]:
print(X_train.shape)
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column >0])

(517788, 144)
id                       517788
member_id                517788
emp_title                 34051
emp_length                31300
url                      517788
                          ...  
settlement_status        498528
settlement_date          498528
settlement_amount        498528
settlement_percentage    498528
settlement_term          498528
Length: 104, dtype: int64


In [4]:
num_cols_with_missing = (missing_val_count_by_column >50000).sum()
num_cols_with_missing

59

In [5]:
# Identify columns with more than 50,000 missing values
cols_to_drop = missing_val_count_by_column[missing_val_count_by_column > 50000].index

# Ensure that the columns to drop actually exist in X_train
cols_to_drop = [col for col in cols_to_drop if col in X_train.columns]

# Drop columns from X_train
X_train.drop(cols_to_drop, axis=1, inplace=True)

# Print the shape of X_train after dropping columns
print(X_train.shape)

(517788, 85)


In [8]:
X_test = pd.read_csv('/kaggle/input/debt-default-prediction/valid.csv')
X_test.dropna(axis=0, subset=['loan_status'], inplace=True)

y_test = X_test.loan_status
X_test.drop(['loan_status'], axis = 1,inplace= True )
train_columns  = X_train.columns
X_test = X_test[train_columns]
X_test.shape

(172596, 85)

* **Let us handle the rest of the missing values with imputations , numerical values with the mean of the column and the object columns with the most frequent value.**

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer()
numerical_cols = [cname for cname in X_train.columns if 
                X_train[cname].dtype in ['int64', 'float64']]

categorical_cols = [cname for cname in X_train.columns if 
                    X_train[cname].dtype == 'object']

numerical_imputer = SimpleImputer(strategy='mean')
X_train_numerical = X_train[numerical_cols].copy()
X_test_numerical = X_test[numerical_cols].copy()
X_train_numerical = numerical_imputer.fit_transform(X_train_numerical)
X_test_numerical = numerical_imputer.transform(X_test_numerical)

# Preprocessing for categorical data
categorical_imputer = SimpleImputer(strategy='most_frequent')
X_train_categorical = X_train[categorical_cols].copy()
X_test_categorical = X_test[categorical_cols].copy()
X_train_categorical = categorical_imputer.fit_transform(X_train_categorical)
X_test_categorical = categorical_imputer.transform(X_test_categorical)

# Convert back to DataFrame

X_train_numerical = pd.DataFrame(X_train_numerical, columns=numerical_cols)
X_train_categorical = pd.DataFrame(X_train_categorical, columns=categorical_cols)
X_test_numerical = pd.DataFrame(X_test_numerical, columns=numerical_cols)
X_test_categorical = pd.DataFrame(X_test_categorical, columns=categorical_cols)


# Now you can combine the numerical and categorical data
X_train = pd.concat([X_train_numerical, X_train_categorical], axis=1)
X_test =  pd.concat([X_test_numerical, X_test_categorical], axis=1)


In [11]:
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column >0])
missing_val_count_by_column_test = (X_test.isnull().sum())
print(missing_val_count_by_column_test[missing_val_count_by_column_test >0])
print(X_train.shape, X_test.shape)

Series([], dtype: int64)
Series([], dtype: int64)
(517788, 85) (172596, 85)
